In [4]:
import json
import utils.qpl.paths as p

OUTPUTS_DIR = p.TYPES_OUTPUT_DIR

def accuracy(data):
    correct = 0
    for d in data:
        if d['actual_type'].strip().lower() == d['predicted_type'].strip().lower():
            correct += 1
    return correct / len(data) if data else 1
            

for output_file in OUTPUTS_DIR.iterdir():
    if output_file.suffix == '.json':
        data = json.loads(output_file.read_text())
        acc = accuracy(data)
        print(f"{output_file.name}: {acc:.2%} ({len(data)} samples)")

old_concert_singer__openai-gpt-4o_seed42_sample20_train_ratio0.5_fewshot2.json: 80.00% (10 samples)
old_concert_singer__openai-gpt-4.1-mini_seed42_sample-1_train_ratio0_fewshot0.json: 69.70% (132 samples)
old_concert_singer__openai-gpt-4.1-mini_seed42_sample10_train_ratio0_fewshot0.json: 90.00% (10 samples)
new_concert_singer__openai-gpt-4.1-mini_seed42_sample-1_train_ratio0.1_fewshot3.json: 84.87% (119 samples)
new_concert_singer__openai-gpt-4.1-mini_seed42_sample-1_train_ratio0_fewshot0.json: 70.45% (132 samples)
old_concert_singer__openai-gpt-4o-mini_seed42_sample20_train_ratio0.5_fewshot2.json: 60.00% (10 samples)
old_concert_singer__openai-gpt-4.1-mini_seed42_sample-1_train_ratio0.1_fewshot3.json: 76.47% (119 samples)
new_concert_singer__openai-gpt-4.1-mini_seed42_sample-1_train_ratio0.15_fewshot3_cot.json: 90.27% (113 samples)
